<a href="https://colab.research.google.com/github/stellaevat/ontology-mapping/blob/main/colabs/bi_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets evaluate

In [ ]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

In [ ]:
import gc
import random
import evaluate
import torch
import numpy as np
from pprint import pprint
from tqdm import tqdm
from sklearn.model_selection import train_test_split, ParameterGrid
from datasets import Dataset, DatasetDict
from transformers import get_scheduler, AutoModel
from transformers.modeling_outputs import SequenceClassifierOutput
from torch.utils.data import DataLoader
from torch.optim import AdamW

# Collate input dataset

In [ ]:
def read_bi_tokenized_from_file(filepath):
  s_input_ids, s_token_type_ids, s_attention_mask, t_input_ids, t_token_type_ids, t_attention_mask, labels = [], [], [], [], [], [], []
  with open(filepath) as f:
    for line in f:
      strings = line.strip().split("],")
      lists = [list(map(int, s.strip("[").split(","))) for s in strings[:-1]]
      s_input_id, s_token_type_id, s_attention, t_input_id, t_token_type_id, t_attention = lists
      label = int(strings[-1])

      s_input_ids.append(s_input_id)
      s_token_type_ids.append(s_token_type_id)
      s_attention_mask.append(s_attention)
      t_input_ids.append(t_input_id)
      t_token_type_ids.append(t_token_type_id)
      t_attention_mask.append(t_attention)
      labels.append(label)

  source_tokenized = {"input_ids" : s_input_ids,
                      "token_type_ids" : s_token_type_ids,
                      "attention_mask" : s_attention_mask}
  target_tokenized = {"input_ids" : t_input_ids,
                      "token_type_ids" : t_token_type_ids,
                      "attention_mask" : t_attention_mask}
  return source_tokenized, target_tokenized, labels

def read_cross_tokenized_from_file(filepath):
  input_ids, token_type_ids, attention_mask, labels = [], [], [], []
  with open(filepath) as f:
    for line in f:
      strings = line.strip().split("],")
      lists = [list(map(int, s.strip("[").split(","))) for s in strings[:-1]]
      input_id, token_type_id, attention = lists
      label = int(strings[-1])

      input_ids.append(input_id)
      token_type_ids.append(token_type_id)
      attention_mask.append(attention)
      labels.append(label)

  tokenized = {"input_ids" : input_ids,
               "token_type_ids" : token_type_ids,
               "attention_mask" : attention_mask}
  return tokenized, labels

def onto_cross_tokenized_from_file(filepath):
  input_ids, token_type_ids, attention_mask = [], [], []
  with open(filepath) as f:
    for line in f:
      strings = line.strip().split("],")
      lists = [list(map(int, s.strip("[").split(","))) for s in strings]
      input_id, token_type_id, attention = lists

      input_ids.append(input_id)
      token_type_ids.append(token_type_id)
      attention_mask.append(attention)

  tokenized = {"input_ids" : input_ids,
               "token_type_ids" : token_type_ids,
               "attention_mask" : attention_mask}
  return tokenized

In [ ]:
def filter_source_target(Xi, source, target):
  X_source = {k : [v[i] for i in Xi] for (k, v) in source.items()}
  X_target = {k : [v[i] for i in Xi] for (k, v) in target.items()}
  return X_source, X_target


def collate_dataset(X_train, X_val, X_test, y_train, y_val, y_test, Xi_train, Xi_val, Xi_test):
  dataset_train = Dataset.from_dict(X_train | {'labels' : y_train, "indices" : Xi_train})
  dataset_val = Dataset.from_dict(X_val | {'labels' : y_val, "indices" : Xi_val})
  dataset_test = Dataset.from_dict(X_test | {'labels' : y_test, "indices" : Xi_test})
  dataset = DatasetDict({
      'train' : dataset_train,
      'val' : dataset_val,
      'test' : dataset_test
  })
  dataset.set_format(type="torch")
  return dataset


def get_bi_datasets_from_tokenized(source_tokenized, target_tokenized, labels):
  Xi = np.arange(len(labels))
  y = np.array(labels)
  Xi_train_val, Xi_test, y_train_val, y_test = train_test_split(Xi, y, test_size=0.2, random_state=3)
  Xi_train, Xi_val, y_train, y_val = train_test_split(Xi_train_val, y_train_val, test_size=0.25, random_state=3)

  X_source_train, X_target_train = filter_source_target(Xi_train, source_tokenized, target_tokenized)
  X_source_val, X_target_val = filter_source_target(Xi_val, source_tokenized, target_tokenized)
  X_source_test, X_target_test = filter_source_target(Xi_test, source_tokenized, target_tokenized)

  print(f"Train: {len(y_train)}")
  print(f"Validation: {len(y_val)}")
  print(f"Test: {len(y_test)}")

  source_data = collate_dataset(X_source_train, X_source_val, X_source_test, y_train, y_val, y_test, Xi_train, Xi_val, Xi_test)
  target_data = collate_dataset(X_target_train, X_target_val, X_target_test, y_train, y_val, y_test, Xi_train, Xi_val, Xi_test)
  return source_data, target_data

# Bi-encoder

In [ ]:
def full_determinism(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.use_deterministic_algorithms(True)
  torch.backends.cudnn.deterministic = True

In [ ]:
class BiEncoderForSequenceClassification(torch.nn.Module):
  def __init__(self, model_name, num_labels, id2label=None, label2id=None, token_embeddings_size=None, hidden_layer=-1):
    super().__init__()
    self.source_model = AutoModel.from_pretrained(model_name)
    self.target_model = AutoModel.from_pretrained(model_name)
    if token_embeddings_size:
      self.source_model.resize_token_embeddings(token_embeddings_size)
      self.target_model.resize_token_embeddings(token_embeddings_size)

    self.num_labels = num_labels
    self.hidden_layer = hidden_layer

    self.sigmoid = torch.nn.Sigmoid()
    self.dropout = torch.nn.Dropout(0.1)
    self.similarity = torch.nn.CosineSimilarity(dim=-1)
    self.loss = torch.nn.BCEWithLogitsLoss()

  def forward(
      self,
      s_input_ids=None, t_input_ids=None,
      s_attention_mask=None, t_attention_mask=None,
      s_token_type_ids=None, t_token_type_ids=None,
      s_position_ids=None, t_position_ids=None,
      s_head_mask=None, t_head_mask=None,
      s_inputs_embeds=None, t_inputs_embeds=None,
      labels=None
    ):

    source_outputs = self.source_model(
      s_input_ids,
      attention_mask=s_attention_mask,
      token_type_ids=s_token_type_ids,
      position_ids=s_position_ids,
      head_mask=s_head_mask,
      inputs_embeds=s_inputs_embeds,
    )

    target_outputs = self.target_model(
      t_input_ids,
      attention_mask=t_attention_mask,
      token_type_ids=t_token_type_ids,
      position_ids=t_position_ids,
      head_mask=t_head_mask,
      inputs_embeds=t_inputs_embeds,
    )

    pooled_source_outputs = self.dropout(source_outputs[1])
    pooled_target_outputs = self.dropout(target_outputs[1])

    # Dot product of source and corresponding target embeddings
    logits = torch.sum(pooled_source_outputs * pooled_target_outputs, dim=-1)

    loss = None
    if labels is not None:
      loss = self.loss(logits.view(-1), labels.view(-1).float())

    return SequenceClassifierOutput(loss=loss, logits=logits)

  def get_source_model_outputs(self, input_ids=None, token_type_ids=None, attention_mask=None, labels=None):
    source_outputs = self.source_model(
      input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids
    )
    return source_outputs[1]

  def get_target_model_outputs(self, input_ids=None, token_type_ids=None, attention_mask=None, labels=None):
    target_outputs = self.target_model(
      input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids
    )
    return target_outputs[1]

In [ ]:
class EarlyStopper:
  def __init__(self, learning_rate, batch_size, patience=1, delta=0):
    self.patience = patience
    self.delta = delta
    self.counter = 0
    self.min_loss = float('inf')

    self.best_epoch = 0
    self.best_metrics = {}
    self.best_model_state = None
    self.best_optimizer_state = None

    self.learning_rate = learning_rate
    self.batch_size = batch_size

  def early_stop(self, loss, epoch, model_state, optimizer_state, metrics):
    if loss < self.min_loss:
      self.min_loss = loss
      self.counter = 0

      self.best_epoch = epoch
      self.best_metrics = metrics
      self.best_model_state = model_state
      self.best_optimizer_state = optimizer_state

    elif loss >= (self.min_loss + self.delta):
      self.counter += 1
      if self.counter >= self.patience:
        return True
    return False

  def save_best_performance(self, filepath):
    performance = f"{self.batch_size},{self.learning_rate},{self.best_epoch},{self.min_loss},{self.best_metrics.get('accuracy')},{self.best_metrics.get('precision')},{self.best_metrics.get('recall')},{self.best_metrics.get('f1')}\n"
    print(performance)
    with open(filepath, 'a') as f:
      f.write(performance)

  def save_best_checkpoint(self, filepath):
    torch.save({
      'model_state_dict' : self.best_model_state,
      'optimizer_state_dict' : self.best_optimizer_state,
      'batch_size' : self.batch_size,
      'learning_rate': self.learning_rate,
      'epoch' : self.best_epoch,
      'loss' : self.min_loss,
      'accuracy' : self.best_metrics.get('accuracy'),
      'precision' : self.best_metrics.get('precision'),
      'recall' : self.best_metrics.get('recall'),
      'f1' : self.best_metrics.get('f1'),
      }, filepath)

  def restore_model_state(self):
    return self.best_model_state

In [ ]:
def show_results(epoch, loss, metrics):
  print(f"\n\nEPOCH {epoch}\n")
  print(f"Training loss: {loss}")
  pprint(metrics)
  print()


def evaluate_biencoder(model, source_data, target_data, batch_size=32, testing=False, verbose=False):
  split = "test" if testing else "val"
  eval_dataloader_index = DataLoader(Dataset.from_dict({'index' : range(len(source_data[split]))}), batch_size=batch_size)
  metrics = [evaluate.load('accuracy'), evaluate.load('precision'), evaluate.load('recall'), evaluate.load('f1')]
  all_indices, all_predictions = [], []

  model.eval()
  avg_loss = 0
  for batch in eval_dataloader_index:
    batch_index = list(batch["index"])
    source_batch = source_data[split][batch_index]
    target_batch = target_data[split][batch_index]
    labels = source_batch["labels"]
    indices = source_batch["indices"]

    source_batch = {"s_" + k: v.to(device) for (k, v) in source_batch.items() if k not in ["labels", "indices"]}
    target_batch = {"t_" + k: v.to(device) for (k, v) in target_batch.items() if k not in ["labels", "indices"]}
    params = source_batch | target_batch
    params["labels"] = labels.to(device)

    with torch.no_grad():
      outputs = model(**params)

    logits = outputs.logits.cpu()
    loss = outputs.loss.item()
    avg_loss += loss * len(batch_index)

    predictions = np.where(logits.squeeze() >= 0.5, 1, 0)
    for metric in metrics:
      metric.add_batch(predictions=predictions, references=labels.cpu())

    all_indices.extend(indices.tolist())
    all_predictions.extend(predictions.tolist())

  avg_loss = avg_loss / len(source_data[split])
  metric_dict = {("Test loss" if testing else "Validation loss") : avg_loss}
  metric_dict.update(metrics[0].compute())
  for metric in metrics[1:]:
    metric_dict.update(metric.compute(average='macro'))

  if verbose:
    pprint(metric_dict)

  if testing:
    return metric_dict, list(zip(all_indices, all_predictions))

  return metric_dict


def train_biencoder(model, source_data, target_data,
                    epochs=10, batch_size=32, learning_rate=1e-5,
                    save_performance=False, save_checkpoint=False,
                    early_stopping=True,
                    verbose=True):
  train_dataloader_index = DataLoader(Dataset.from_dict({'index' : range(len(source_data["train"]))}), shuffle=True, batch_size=batch_size)
  num_training_steps = epochs * len(train_dataloader_index)

  optimizer = AdamW(model.parameters(), lr=learning_rate)
  scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
  early_stopper = EarlyStopper(patience=3, learning_rate=learning_rate, batch_size=batch_size)

  progress_bar = tqdm(range(num_training_steps), position=0, leave=True)

  model.train()
  for epoch in range(1, epochs+1):
    avg_loss = 0
    for batch in train_dataloader_index:
      batch_index = list(batch["index"])
      source_batch = source_data["train"][batch_index]
      target_batch = target_data["train"][batch_index]
      labels = source_batch["labels"]

      source_batch = {"s_" + k: v.to(device) for (k, v) in source_batch.items() if k not in ["labels", "indices"]}
      target_batch = {"t_" + k: v.to(device) for (k, v) in target_batch.items() if k not in ["labels", "indices"]}
      params = source_batch | target_batch
      params["labels"] = labels.to(device)

      outputs = model(**params)
      loss = outputs.loss
      avg_loss += loss * len(batch["index"])
      loss.backward()

      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      progress_bar.update(1)

    avg_loss = avg_loss / len(source_data["train"])
    metrics = evaluate_biencoder(model, source_data, target_data, batch_size=batch_size)
    if verbose:
      show_results(epoch, avg_loss, metrics)

    val_loss = metrics["Validation loss"]
    if early_stopper.early_stop(val_loss, epoch, model.state_dict(), optimizer.state_dict(), metrics) and early_stopping:
      break

  model.load_state_dict(early_stopper.restore_model_state())

  if save_performance:
    early_stopper.save_best_performance(save_performance)
  if save_checkpoint:
    early_stopper.save_best_checkpoint(save_checkpoint)

# Hyperparameter tuning

In [ ]:
param_grid = ParameterGrid({
    "batch_size" : [2 ** i for i in range(4, 7)],
    "learning_rate" : [10 ** i for i in range(-6, -2)]
})

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pretrained = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

for params in list(param_grid):
  print(params)

  full_determinism(seed=3)
  model = BiEncoderForSequenceClassification(pretrained, num_labels=1)
  model.to(device)

  save_performance = f"bi_performance_{feature}_{negatives}.csv"
  train_biencoder(
      model, source_data, target_data,
      save_performance=save_performance,
      verbose=True,
      **params
  )

  model.cpu()
  del model
  torch.cuda.empty_cache()
  gc.collect()

# Saving models

In [ ]:
def write_bi_encoder_predictions(predictions, filepath):
  with open(filepath, "w") as f:
    for (i, prediction) in predictions:
      f.write(f"{i},{prediction}\n")

def write_experiment_metrics(experiment, metrics, filepath):
  experiment_str = ",".join(experiment)
  metrics_str = f'{metrics.get("Test loss")},{metrics.get("accuracy")},{metrics.get("precision")},{metrics.get("recall")},{metrics.get("f1")}\n'
  entry = ",".join([experiment_str, metrics_str])
  with open(filepath, 'a') as f:
    f.write(entry)

In [ ]:
tuned_params = {
    ("term", "random")    : {"batch_size" : 16, "learning_rate" : 1e-05},  # 4 epochs
    ("term", "multi")     : {"batch_size" : 16, "learning_rate" : 1e-05},  # 4 epochs
    ("term", "neighbour") : {"batch_size" : 32, "learning_rate" : 1e-05},  # 4 epochs
    ("ext", "random")     : {"batch_size" : 16, "learning_rate" : 1e-05},  # 5 epochs
    ("ext", "multi")      : {"batch_size" : 32, "learning_rate" : 1e-05},  # 7 epochs
    ("ext", "neighbour")  : {"batch_size" : 64, "learning_rate" : 1e-05},  # 2 epochs
}

features = ['term', 'int', 'ext']
negative_sampling = ['random', 'multi', 'neighbour']
direction = "ncit2doid"

feature = features[2]
negatives = negative_sampling[2]
load_from_file = False

source_tokenized, target_tokenized, labels = read_bi_tokenized_from_file(f"bi_tokenized_{feature}_{negatives}_{direction}.csv")
source_data, target_data = get_bi_datasets_from_tokenized(source_tokenized, target_tokenized, labels)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pretrained = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

full_determinism(seed=3)
model = BiEncoderForSequenceClassification(pretrained, num_labels=1)
if load_from_file:
  checkpoint = torch.load(f"bi_checkpoint_{feature}_{negatives}.csv")
  model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

experiment = (feature, negatives)
params = tuned_params[experiment]

if not load_from_file:
  save_checkpoint = f"bi_checkpoint_{feature}_{negatives}.csv"
  train_biencoder(
      model, source_data, target_data,
      save_checkpoint=save_checkpoint,
      verbose=False,
      **params
  )

In [ ]:
full_determinism(seed=3)

metrics, predictions = evaluate_biencoder(
    model, source_data, target_data,
    batch_size=params["batch_size"],
    testing=True,
    verbose=True
)

write_bi_encoder_predictions(predictions, f"bi_predictions_{feature}_{negatives}.csv")
write_experiment_metrics(experiment, metrics, "bi_test_metrics.csv")

In [ ]:
model.cpu()
del model
torch.cuda.empty_cache()
gc.collect()